<a href="https://colab.research.google.com/github/butchland/fastai_xla_extensions/blob/master/explore_nbs/fast_multi_tpu_mnist_benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uqq cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp36-cp36m-linux_x86_64.whl

     |████████████████████████████████| 133.6MB 31kB/s 
     |████████████████████████████████| 61kB 3.6MB/s 


In [ ]:
# !pip install -Uqq git+https://github.com/fastai/fastai.git 
!pip install -Uqq fastai --upgrade

     |████████████████████████████████| 194kB 5.9MB/s 
     |████████████████████████████████| 61kB 5.3MB/s 


In [ ]:
!pip install -Uqq git+https://github.com/butchland/fastai_xla_extensions.git

In [ ]:
!pip install -Uqq git+https://github.com/butchland/my_timesaver_utils.git

In [ ]:
!curl -s https://course19.fast.ai/setup/colab | bash

Updating fastai...
Done.


In [ ]:
!pip freeze | grep torch
!pip freeze | grep fast

torch==1.7.0+cu101
torch-xla==1.7
torchsummary==1.5.1
torchtext==0.3.1
torchvision==0.8.1+cu101
fastai==2.2.5
fastai-xla-extensions==0.0.7
fastcore==1.3.19
fastdtw==0.3.4
fastprogress==1.0.0
fastrlock==0.5


Start of kernel

In [ ]:
from fastai.vision.all import *
from fastai_xla_extensions.multi_core import *
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp
from my_timesaver_utils.profiling import *
import torch

In [ ]:
FLAGS = {}
FLAGS['num_cores'] = 8 if os.environ.get('TPU_NAME', False) else 1
FLAGS['sync_valid'] = True
FLAGS['batch_size'] = 256
FLAGS['image_size'] = 28
FLAGS['momentum'] = (0.95,0.85,0.95)
FLAGS['weight_decay'] = None
FLAGS['learning_rate'] = 2e-3
FLAGS['num_epochs'] = 5
FLAGS['num_workers'] = 4


In [ ]:
ARCH = resnet34

In [ ]:
PATH = untar_data(URLs.MNIST)

In [ ]:
fname_labeller = parent_label 
# fname_labeller(get_image_files(PATH)[0]) # trigger regex compilation of pat


In [ ]:
splitter = GrandparentSplitter(train_name='training', valid_name='testing')

In [ ]:

DATA = DataBlock(
    blocks=(ImageBlock,CategoryBlock),
    get_items=get_image_files,
    get_y=fname_labeller,
    splitter=splitter,
    item_tfms=Resize(FLAGS['image_size']),
    batch_tfms=[]
)

In [ ]:
DLS = DATA.dataloaders(PATH, bs=FLAGS['batch_size'], device=None)
vocab = CategoryMap(get_image_files(PATH).map(fname_labeller))
N_OUT = len(vocab)

In [ ]:
import torchvision as thv
pil2tensor = thv.transforms.ToTensor()
resize_tfm = thv.transforms.Resize((FLAGS['image_size'],FLAGS['image_size']))
norm = thv.transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))
x_tfms=[resize_tfm,pil2tensor,norm,], 
y_tfms=[fname_labeller,VocabularyMapper(),],


DSET_BUILDER = TorchDatasetBuilder(
    PATH, 
    get_items=get_image_files, 
    splitter=splitter,
    x_tfms=x_tfms,
    y_tfms=y_tfms,
    x_type_tfms=PILImage.create,
)


In [ ]:
DSET_BUILDER.x_tfms = DSET_BUILDER.x_tfms[0]

In [ ]:
DSET_BUILDER.y_tfms = DSET_BUILDER.y_tfms[0]

In [ ]:

DSET_BUILDER.setup(get_image_files(PATH),do_setup=True)
# DSETS = DSET_BUILDER.get_datasets()

In [ ]:
# DSETS = DSET_BUILDER.get_datasets()

In [ ]:
DSET_BUILDER.x_tfms

[Resize(size=(28, 28), interpolation=PIL.Image.BILINEAR),
 ToTensor(),
 Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.201))]

In [ ]:
DSET_BUILDER.y_tfms

[<function fastai.data.transforms.parent_label>,

In [ ]:
len(DSET_BUILDER.y_tfms)

2

In [ ]:
DSET_BUILDER.y_tfms[1].vocab

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [ ]:
custom_model = create_cnn_model(ARCH, N_OUT, pretrained=True, concat_pool=False)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


In [ ]:
WRAPPED_MODEL = xmp.MpModelWrapper(custom_model)

In [ ]:
LOSS_FUNC = nn.CrossEntropyLoss()
OPT_FUNC = Adam

In [ ]:
def run_simple(rank, flags):
    device = xm.xla_device()
    # print(f'xla: {rank} device: {device}')
    tlist = []
    for i in range(5):
        t1 = torch.tensor(rank).float()
        t1 = t1.to(device)
        tlist.append(t1 * (i+1.))
    print(f'xla: {rank} t1 {t1.clone().detach().item()} device: {t1.device}')
    tlist2 = xm.all_reduce(xm.REDUCE_SUM,tlist)
    ptlist2 = [t.clone().detach().item() for t in tlist2]
    print(f'xla:{rank} ptlist2: {ptlist2}')


In [ ]:
def time_train_model(rank, flags):
    print(f'xla: {rank} start time train model')
    device = xm.xla_device()
    world_size = xm.xrt_world_size()
    bs = flags['batch_size']
    sync_valid = flags['sync_valid']
    num_workers = flags['num_workers']
    t1 = torch.tensor(rank).float()
    t1 = t1.to(device)  
    # print(f'xla: {rank} t1 {t1.clone().detach().item()} device: {t1.device}')

    dload_rec = f'make_dload{rank}'
    start_record(dload_rec)
    # dls = DATA.dataloaders(PATH, bs=bs)
    dsets = DSET_BUILDER.get_datasets()
    dls = make_torch_dataloaders(
        *dsets,
        rank,
        world_size,
        bs=bs,
        num_workers=num_workers,
        sync_valid=sync_valid
    )
    # dls = make_distributed_dataloaders(DLS, rank, world_size, sync_valid=sync_valid)
    # make_distributed_dataloaders(dls, rank, world_size, sync_valid=False):
    # dls = make_fastai_dataloaders(DATA,PATH, rank, world_size, bs=bs, sync_valid=FLAGS['sync_valid'])
    # make_fastai_dataloaders(datablock, source, rank, world_size, device=None, path='.', sync_valid=False, verbose=False,**kwargs):
    end_record(dload_rec)
    print_prof_data(dload_rec)

    dload_rec2 = f'build_learner{rank}'
    start_record(dload_rec2)
    model = WRAPPED_MODEL.to(device)
    moms = flags['momentum']
    wd = flags['weight_decay']
    learner = Learner(dls, model, loss_func=LOSS_FUNC, opt_func=OPT_FUNC,
                      metrics=accuracy, wd=wd, moms=moms)
    learner.to_xla(device, rank, sync_valid=sync_valid)
    end_record(dload_rec2)
    print_prof_data(dload_rec2)

    dload_rec3 = f'fit_learner{rank}'
    start_record(dload_rec3)

    epochs = flags['num_epochs']
    lr_max = flags['learning_rate']
    learner.fit_one_cycle(epochs, lr_max=lr_max)

    end_record(dload_rec3)
    print_prof_data(dload_rec3)

    t2 = xm.all_reduce(xm.REDUCE_SUM,t1)
    t3 = t2.clone().detach().item()
    # print(f'xla:{rank} t2: {t2.clone().detach().item()}')
    xm.mark_step()
    print(f'xla: {rank} time train model completed')


In [ ]:
def time_dataloaders(rank, flags):
    print(f'xla: {rank} start time dataloaders')
    device = xm.xla_device()
    world_size = xm.xrt_world_size()
    bs = flags['batch_size']
    sync_valid = flags['sync_valid']

    t1 = torch.tensor(rank).float()
    t1 = t1.to(device)  
    dload_rec = f'make_dload{rank}'
    start_record(dload_rec)
    # dls = DATA.dataloaders(PATH, bs=bs)
    dls = make_fastai_dataloaders(DATA,PATH, rank, world_size, bs=bs, sync_valid=FLAGS['sync_valid'])
    # make_fastai_dataloaders(datablock, source, rank, world_size, device=None, path='.', sync_valid=False, verbose=False,**kwargs):
    end_record(dload_rec)
    print_prof_data(dload_rec)

    # dload_rec2 = f'make_distrib_dload{rank}'
    # start_record(dload_rec2)
    # dls = make_distributed_dataloaders(DLS, rank, world_size, sync_valid=sync_valid)
    # print(f'xla: {rank} t1 {t1.clone().detach().item()} device: {t1.device}')
    # end_record(dload_rec2)
    # print_prof_data(dload_rec2)
    t2 = xm.all_reduce(xm.REDUCE_SUM,t1)
    t3 = t2.clone().detach().item()
    # print(f'xla:{rank} t2: {t2.clone().detach().item()}')
    xm.mark_step()
    print(f'xla: {rank} time dataloaders completed')


In [ ]:
%%time
xmp.spawn(time_train_model, 
          args=(FLAGS,),
          nprocs=FLAGS['num_cores'],
          start_method='fork')

xla: 0 start time train model
xla: 4 start time train model
xla: 5 start time train model
Function make_dload0 called 1 times.
Execution time max: 1.977, average: 1.977
Function make_dload4 called 1 times.
Execution time max: 1.962, average: 1.962
Function make_dload5 called 1 times.
Execution time max: 1.961, average: 1.961
Function build_learner0 called 1 times.
Execution time max: 1.416, average: 1.416
start fit


epoch,train_loss,valid_loss,accuracy,time
0,0.327332,0.132322,0.968872,00:44
1,0.207357,0.026093,0.991943,00:24
2,0.127972,0.024927,0.991943,00:08
3,0.076895,0.017891,0.994873,00:08
4,0.047691,0.017472,0.994507,00:07


Function build_learner4 called 1 times.
Execution time max: 2.739, average: 2.739
xla: 2 start time train model
Function build_learner5 called 1 times.
Execution time max: 3.934, average: 3.934
xla: 6 start time train model
xla: 1 start time train model
xla: 3 start time train model
xla: 7 start time train model
Function make_dload2 called 1 times.
Execution time max: 1.991, average: 1.991
Function make_dload6 called 1 times.
Execution time max: 2.038, average: 2.038
Function make_dload1 called 1 times.
Execution time max: 2.013, average: 2.013
Function make_dload3 called 1 times.
Execution time max: 1.984, average: 1.984
Function make_dload7 called 1 times.
Execution time max: 1.993, average: 1.993
Function build_learner2 called 1 times.
Execution time max: 1.440, average: 1.440
Function build_learner6 called 1 times.
Execution time max: 2.452, average: 2.452
Function build_learner1 called 1 times.
Execution time max: 3.746, average: 3.746
Function build_learner3 called 1 times.
Execu